In [2]:
import cv2
import os
import torch
import pandas as pd
from PIL import Image
from datetime import datetime
import time
from collections import defaultdict
from facenet_pytorch import MTCNN, InceptionResnetV1

# Excel file setup
EXCEL_FILE = 'attendance.xlsx'
if not os.path.exists(EXCEL_FILE):
    df_init = pd.DataFrame(columns=["Subject", "Entry Time", "Attendance", "Mask", "On Time", "Person Name", "Date"])
    df_init.to_excel(EXCEL_FILE, index=False, engine='openpyxl')

# Clear records function
def clear_attendance_records():
    df = pd.DataFrame(columns=["Subject", "Entry Time", "Attendance", "Mask", "On Time", "Person Name", "Date"])
    df.to_excel(EXCEL_FILE, index=False, engine='openpyxl')
    print("Attendance records cleared.")

# Add new student
def add_new_student():
    new_name = input("Enter new student's name (e.g., JohnDoe): ").strip()
    known_faces_dir = "known_faces"
    student_path = os.path.join(known_faces_dir, new_name)

    if os.path.exists(student_path):
        print("Folder already exists for this student.")
    else:
        os.makedirs(student_path)
        print(f"Folder created at: {student_path}")
        print("Please add at least 5 clear photos of the student (with and without mask) to this folder.")
        print("Once done, restart the program and select 'Take Attendance'.")

# User input menu
while True:
    print("\nReal-time Attendance System")
    print("1. Take Attendance")
    print("2. Clear Attendance Records")
    print("3. Add New Student")
    choice = input("Enter your choice (1, 2, or 3): ")

    if choice == "2":
        clear_attendance_records()
        exit_choice = input("Do you want to exit? (yes/no): ").strip().lower()
        if exit_choice == 'yes':
            print("Exiting program.")
            exit()
        else:
            continue
    elif choice == "3":
        add_new_student()
        exit_choice = input("Do you want to exit? (yes/no): ").strip().lower()
        if exit_choice == 'yes':
            print("Exiting program.")
            exit()
        else:
            continue
    elif choice == "1":
        break
    else:
        print("Invalid choice. Try again.")

# Timetable
TIMETABLE = {
    'Monday': [(9, 'OS'), (10, 'OS'), (11, 'ML'), (12, 'Lunch'), (13, 'CN'), (14, 'CN'), (15, 'ML')],
    'Tuesday': [(9, 'CN'), (10, 'CN'), (11, 'OS'), (12, 'Lunch'), (13, 'ML'), (14, 'ML'), (15, 'OS')],
    'Wednesday': [(9, 'ML'), (10, 'ML'), (11, 'OS'), (12, 'Lunch'), (13, 'CN'), (14, 'CN'), (15, 'Project')],
    'Thursday': [(9, 'OS'), (10, 'CN'), (11, 'ML'), (12, 'Lunch'), (13, 'Project'), (14, 'Project'), (15, 'Library')],
    'Friday': [(9, 'CN'), (10, 'OS'), (11, 'ML'), (12, 'Lunch'), (13, 'Project'), (14, 'ML'), (15, 'OS')]
}

# Load YOLOv5 model
yolo_model = torch.hub.load('yolov5', 'custom', path='last.pt', source='local')
class_names = ['without_mask', 'with_mask', 'mask_weared_incorrect']

# Setup FaceNet and MTCNN
device = 'cuda' if torch.cuda.is_available() else 'cpu'
mtcnn = MTCNN(image_size=160, margin=20, keep_all=True, device=device)
facenet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Load known faces
def load_known_faces(known_dir='known_faces'):
    embeddings = {}
    for person_name in os.listdir(known_dir):
        person_dir = os.path.join(known_dir, person_name)
        if not os.path.isdir(person_dir):
            continue
        embeddings[person_name] = {'with_mask': [], 'without_mask': []}
        for img_file in os.listdir(person_dir):
            if not img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
            path = os.path.join(person_dir, img_file)
            is_masked = 'mask' in img_file.lower()
            for flip in [False, True]:
                img = Image.open(path).convert('RGB')
                if flip:
                    img = img.transpose(Image.FLIP_LEFT_RIGHT)
                faces = mtcnn(img)
                if faces is not None:
                    face = faces[0] if isinstance(faces, list) else faces
                    if face is not None:
                        face = face.to(device)
                        if len(face.shape) == 4:
                            face = face.squeeze(0)
                        if len(face.shape) == 3:
                            face = face.unsqueeze(0)
                        emb = facenet(face).detach()
                        category = 'with_mask' if is_masked else 'without_mask'
                        embeddings[person_name][category].append(emb)
    return embeddings

print("Loading known faces...")
known_embeddings = load_known_faces()
print(f"Loaded {len(known_embeddings)} persons.")

# Recognize face
def recognize_face(face_embedding, threshold=0.9):
    min_dist = float('inf')
    identity = "Unknown"
    for person_name, emb_dict in known_embeddings.items():
        for compare_type in ['with_mask', 'without_mask']:
            for known_emb in emb_dict[compare_type]:
                dist = (face_embedding - known_emb).norm().item()
                if dist < min_dist:
                    min_dist = dist
                    identity = person_name
    return identity if min_dist < threshold else "Unknown", min_dist

# Get current subject from timetable
def get_current_subject():
    now = datetime.now()
    weekday = now.strftime('%A')
    current_hour = now.hour
    schedule = TIMETABLE.get(weekday, [])
    for hour, subject in schedule:
        if current_hour == hour:
            return subject, hour
    return None, None

# Log attendance
def log_attendance(name, mask_status, is_present=True):
    if name == "Unknown":
        return

    now = datetime.now()
    subject, subject_hour = get_current_subject()
    if subject is None:
        return

    entry_time_str = now.strftime("%H:%M:%S")
    df = pd.read_excel(EXCEL_FILE, engine='openpyxl')

    class_start_time = now.replace(hour=subject_hour, minute=0, second=0, microsecond=0)
    is_on_time = (now - class_start_time).seconds <= 600

    new_row = {
        "Subject": subject,
        "Entry Time": entry_time_str if is_present else "N/A",
        "Attendance": "Present" if is_present else "Absent",
        "Mask": mask_status if is_present else "N/A",
        "On Time": "Yes" if is_present and is_on_time else "No",
        "Person Name": name,
        "Date": now.strftime("%Y-%m-%d")
    }

    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    df.to_excel(EXCEL_FILE, index=False, engine='openpyxl')

# Webcam setup
cap = cv2.VideoCapture(0)
print("Webcam started. Press 'q' to quit.")

recognized_persons = set()
last_logged_times = defaultdict(lambda: 0)
ATTENDANCE_INTERVAL = 600  # seconds (10 minutes)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.flip(frame, 1)
    display_frame = frame.copy()

    results = yolo_model(frame)
    detections = results.xyxy[0]

    for det in detections:
        x1, y1, x2, y2, conf, cls = det.cpu().numpy()
        x1, y1, x2, y2, cls = map(int, [x1, y1, x2, y2, cls])
        if conf < 0.5:
            continue

        face_crop = frame[y1:y2, x1:x2]
        if face_crop.size == 0:
            continue

        face_rgb = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)
        face_pil = Image.fromarray(face_rgb)
        face_pil_flipped = face_pil.transpose(Image.FLIP_LEFT_RIGHT)

        identity = "Unknown"
        min_distance = float('inf')
        original_mask_status = class_names[cls]

        if original_mask_status == 'without_mask' and (conf * 100) > 91.8:
            custom_mask_status = "with_mask"
        else:
            custom_mask_status = original_mask_status

        for face_img in [face_pil, face_pil_flipped]:
            face_tensor = mtcnn(face_img)
            if face_tensor is not None:
                face_tensor = face_tensor[0] if isinstance(face_tensor, list) else face_tensor
                if face_tensor is not None:
                    face_tensor = face_tensor.to(device)
                    if len(face_tensor.shape) == 4:
                        face_tensor = face_tensor.squeeze(0)
                    if len(face_tensor.shape) == 3:
                        face_tensor = face_tensor.unsqueeze(0)
                    emb = facenet(face_tensor)
                    name, distance = recognize_face(emb)
                    if distance < min_distance:
                        identity = name
                        min_distance = distance

        if identity != "Unknown":
            current_time = time.time()
            if (current_time - last_logged_times[identity]) > ATTENDANCE_INTERVAL:
                recognized_persons.add(identity)
                last_logged_times[identity] = current_time
                log_attendance(identity, custom_mask_status)

        color = (0, 255, 0) if custom_mask_status == "with_mask" else (0, 165, 255) if custom_mask_status == "mask_weared_incorrect" else (0, 0, 255)
        label = f"{identity} - {custom_mask_status} ({min_distance:.2f})"
        cv2.rectangle(display_frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(display_frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    if len(detections) > 0:
        cv2.putText(display_frame, f"Detections: {len(detections)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    cv2.imshow("Face & Mask Recognition", display_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()

# Mark unrecognized known persons as absent
for person_name in known_embeddings.keys():
    if person_name not in recognized_persons:
        log_attendance(person_name, mask_status="N/A", is_present=False)

cv2.destroyAllWindows()



Real-time Attendance System
1. Take Attendance
2. Clear Attendance Records
3. Add New Student


YOLOv5  v7.0-411-gf4d8a84c Python-3.12.7 torch-2.2.2+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Loading known faces...
Loaded 3 persons.
Webcam started. Press 'q' to quit.
